# TESTS

In [1]:
from pathlib import Path
import os
import modflowapi
from modflowapi import Callbacks
from workflow import *
from datetime import datetime

DT_FMT = "%Y-%m-%d %H:%M:%S"


In [4]:
sim = build_model(ws = 'model', sim_name = 'engesgaard1992')
# strt


Setting model for component: chloride
--- Building IMS package ---
--- Building DIS package ---
--- Building ADV package ---
--- Building DSP package ---
--- Building MST package ---
building FMI and OC ...
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing solution package ims_0...
  writing package chloride.gwfgwt...
  writing model gwf...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package chd...
    writing package wel...
INFORMATION: maxbound in ('gwf6', 'wel', 'dimensions') changed to 1 based on size of stress_period_data
    writing package oc...
  writing model chloride...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package ssm...
    writing package adv...
    writing package dsp...
    writing package mst...
    writing package oc...


In [ ]:
# sim_ws = os.path.join('model', '1dtest', 'chloride')
sim_ws = Path("model/engesgaard1992/")
dll = Path("bin/win/libmf6")

In [ ]:
def try_get_value(address, mf6):
    try:
        res = mf6.get_value(address)
        return res
    except:
        res = 'failed'
        return res

In [ ]:
mf6 = modflowapi.ModflowApi(dll, working_directory = sim_ws)
mf6.initialize()
# current_time = mf6.get_current_time()
# end_time = mf6.get_end_time()
all_vars = mf6.get_input_var_names() 

# var_add = 

# try:
#     mf6.finalize()
#     success=True
# except:
#     raise RuntimeError

In [ ]:
# current_time, end_time

# all_vars
df_vars = pd.DataFrame(all_vars)[0].str.split('/', expand=True)
df_vars['addr'] = all_vars
df_vars.to_csv('vars.csv')

In [ ]:
val = mf6.get_value(mf6.get_var_address("x", 'SODIUM'))
val += 1

# val = val + 1
val += 1
# mf6.get_value(mf6.get_var_address("x", 'SODIUM')) = val

In [ ]:
mf6.finalize()
val

In [ ]:

mf6 = modflowapi.ModflowApi(dll, working_directory = sim_ws)
mf6.initialize()

sim_start = datetime.now()
print("...starting transport solution at {0}".format(sim_start.strftime(DT_FMT)))
# reset the node tracking containers

# get the current sim time
ctime = mf6.get_current_time()
ctimes = [0.0]
# get the ending sim time
etime = mf6.get_end_time()
# max number of solution iterations
max_iter = mf6.get_value(mf6.get_var_address("MXITER", "SLN_1"))
num_fails = 0
# let's do it!
while ctime < etime:
    sol_start = datetime.now()
    # length of the current solve time
    dt = mf6.get_time_step()
    # prep the current time step
    mf6.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    mf6.prepare_solve(1)
    # the one-based stress period number
    stress_period = mf6.get_value(mf6.get_var_address("KPER", "TDIS"))[0]
    time_step = mf6.get_value(mf6.get_var_address("KSTP", "TDIS"))[0]

    # solve until converged
    while kiter < max_iter:
        # apply whatever change we want here
        # val = mf6.get_value(mf6.get_var_address("x", 'SODIUM'))
        # val += 1
        convg = mf6.solve(1)
        if convg:
            td = (datetime.now() - sol_start).total_seconds() / 60.0
            print("transport stress period,time step {0},{1} converged with {2} iters, took {3:10.5G} mins".format(stress_period, time_step, kiter,td))
            break
        kiter += 1

    if not convg:
        td = (datetime.now() - sol_start).total_seconds() / 60.0
        print("transport stress period,time step {0},{1} did not converged, {2} iters, took {3:10.5G} mins".format(
            stress_period, time_step, kiter, td))
        num_fails += 1
mf6.finalize()


In [ ]:
def callback_function(sim, callback_step):
    """
    A demonstration function that dynamically adjusts recharge
    and pumping in a modflow-6 model through the MODFLOW-API

    Parameters
    ----------
    sim : modflowapi.ApiSimulation
        A simulation object for the solution group that is
        currently being solved
    step : enumeration
        modflowapi.Callbacks enumeration object that indicates
        the part of the solution modflow is currently in.
    """
    ml = sim.get_model()
    if callback_step == Callbacks.initialize:
        print(sim.models)

    # if callback_step == Callbacks.stress_period_start:
    #     # adjust recharge for stress periods 7 through 12
    #     if sim.kper <= 6:
    #         rcha = ml.rcha_0
    #         spd = rcha.stress_period_data
    #         print(f"updating recharge: stress_period={ml.kper}")
    #         spd["recharge"] += 0.40 * sim.kper

    if callback_step == Callbacks.timestep_start:
        print(
            f"Testing: stress_period={ml.kper}, timestep={ml.kstp}", ml.kstp
        )
        if ml.kstp != 2:
            ml.wel.stress_period_data["q"] *= 1/ml.kstp
            # ml.wel.stress_period_data['CONCENTRATION'] = 100000000
            # print('..........................', ml.wel.stress_period_data['CONCENTRATION'])
        else:
            ml.wel.stress_period_data['q'] = 0
        # if ml.kstp ==2:
            
    if callback_step == Callbacks.iteration_start:
        # we can implement complex solutions to boundary conditions here!
        pass
    # return wel

# modflowapi.run_simulation(dll, sim_ws, callback_function, verbose=True)


In [ ]:
ucn = flopy.utils.HeadFile(os.path.join("model",'engesgaard1992',"sodium.ucn"),text="concentration")
carr = ucn.get_alldata()

In [ ]:
# carr[0][0]
carr[-1,0,0,:]

# PHREEQCRM TEST

In [5]:
import phreeqcrm
import numpy as np
import os
import sys

In [6]:
nxyz = 50
nthreads = 1
phreeqc_rm = phreeqcrm.PhreeqcRM(nxyz, nthreads)

In [7]:
status = phreeqc_rm.SetComponentH2O(False)
phreeqc_rm.UseSolutionDensityVolume(False)
status = phreeqc_rm.SetFilePrefix("SimpleAdvect_py")
phreeqc_rm.OpenFiles()
poro = np.full((nxyz), .1)
status = phreeqc_rm.SetPorosity(poro)

In [8]:
poro.shape

(50,)

In [9]:
print_chemistry_mask = np.full((nxyz), 1)
status = phreeqc_rm.SetPrintChemistryMask(print_chemistry_mask)
nchem = phreeqc_rm.GetChemistryCellCount()

In [10]:
# Load database
databasews = os.path.join("database", "pht3d_datab.dat")
status = phreeqc_rm.LoadDatabase(databasews)
status = phreeqc_rm.RunFile(True, True, True, "phinp.dat")
# Clear contents of workers and utility
input = "DELETE; -all"
status = phreeqc_rm.RunString(True, False, True, input)

ncomps = phreeqc_rm.FindComponents()
ncomps

components = phreeqc_rm.GetComponents()
components

for comp in components:
    phreeqc_rm.OutputMessage(comp)
phreeqc_rm.OutputMessage("\n")

ic1 = [-1] * nxyz * 7 
ic1
for i in range(nxyz):
    ic1[i]            =  1  # Solution 1
    ic1[nxyz + i]     = 1  # Equilibrium phases none
    ic1[2 * nxyz + i] =  -1  # Exchange 1
    ic1[3 * nxyz + i] = -1  # Surface none
    ic1[4 * nxyz + i] = -1  # Gas phase none
    ic1[5 * nxyz + i] = -1  # Solid solutions none
    ic1[6 * nxyz + i] = -1  # Kinetics none

status = phreeqc_rm.InitialPhreeqc2Module(ic1)

# Initial equilibration of cells
time = 0.0
time_step = 0.0
status = phreeqc_rm.SetTime(time)
status = phreeqc_rm.SetTimeStep(time_step)
status = phreeqc_rm.RunCells()


In [11]:
status

c_dbl_vect = phreeqc_rm.GetConcentrations()


In [12]:
#ncomp * ncell

ncomps*nxyz, components


(350, ('H', 'O', 'Charge', 'C', 'Ca', 'Cl', 'Mg'))

In [13]:
phreeqc_rm.GetSpeciesNames()
phreeqc_rm.GetSpeciesConcentrations()

array([], dtype=float64)

In [14]:
phreeqc_rm.GetSpeciesNames()


('CH4',
 'CO2',
 'CO3-2',
 'Ca+2',
 'CaCO3',
 'CaHCO3+',
 'CaOH+',
 'Cl-',
 'H+',
 'H2',
 'H2O',
 'H2O-0.01',
 'HCO3-',
 'Mg+2',
 'MgCO3',
 'MgHCO3+',
 'MgOH+',
 'O2',
 'OH-')

In [15]:
phreeqc_rm.GetDatabaseFileName()
components
phreeqc_rm.GetEquilibriumPhases()
# phreeqc_rm.SetConcentrations()

('Calcite', 'Dolomite')

In [16]:
components

('H', 'O', 'Charge', 'C', 'Ca', 'Cl', 'Mg')

In [17]:
# status
c_dbl_vect = phreeqc_rm.GetConcentrations()

In [18]:
nlay = 1  # Number of layers
Lx = 0.5 #m
ncol = 50  # Number of columns
nrow = 1  

n=nxyz

conc = [c_dbl_vect[i:i + n] for i in range(0, len(c_dbl_vect), n)]

sconc = {}
for e, c in enumerate(components):
    sconc[c] = np.reshape(conc[e], (nlay, nrow, ncol))
sconc

for c in components:
    print(sconc[c].dtype)



float64
float64
float64
float64
float64
float64
float64


In [19]:
sim = build_model(ws = 'model', sim_name = 'engesgaard1992', spls = components, sconc=sconc)

# strt

Setting model for component: H
--- Building IMS package ---
--- Building DIS package ---
--- Building ADV package ---
--- Building DSP package ---
--- Building MST package ---
building FMI and OC ...
Setting model for component: O
--- Building IMS package ---
--- Building DIS package ---
--- Building ADV package ---
--- Building DSP package ---
--- Building MST package ---
building FMI and OC ...
Setting model for component: Charge
--- Building IMS package ---
--- Building DIS package ---
--- Building ADV package ---
--- Building DSP package ---
--- Building MST package ---
building FMI and OC ...
Setting model for component: C
--- Building IMS package ---
--- Building DIS package ---
--- Building ADV package ---
--- Building DSP package ---
--- Building MST package ---
building FMI and OC ...
Setting model for component: Ca
--- Building IMS package ---
--- Building DIS package ---
--- Building ADV package ---
--- Building DSP package ---
--- Building MST package ---
building FMI and O

# Engesgaard 1992 - PHT3D

In [ ]:
wd = os.path.join('benchmark', 'pht3d')
dx = 0.01
simdf = pd.read_csv(os.path.join(wd, 'out.dat'), sep = '\t', skipinitialspace=True, index_col=[0])
simdf.drop(simdf.columns[len(simdf.columns)-1], axis=1, inplace=True)
# simdf.columns = [col.lower() for col in simdf.columns]
simdf.loc[:, 'x'] = simdf['cell'] * dx 
simdf.head()


In [ ]:
fig, axs = plt.subplots(4,2, figsize = (6.3, 8.3))
t = 0.24
for var, ax in zip(simdf.iloc[:,1:-1 ].columns, axs.flatten()):
    ax.plot(simdf.x.unique(), simdf.loc[t, var])
    ax.set_xlabel('distance (m)')
    if var not in ['ph', 'pe']:
        ax.set_ylabel('C (mol L$^{-1}$)')
    ax.set_title(f'{var}')

fig.tight_layout()

In [ ]:
simdf.loc[0.24, ['x','calcite']].pivot_table(index='x', columns = 'time_d').plot() 

In [ ]:
simdf.loc[0.24, ['x','mg']].pivot_table(index='x', columns = 'time_d').plot() 